# **实验题目2：美国总统大选数据分析**

实验内容：现有一份美国大选捐款的统计数据，根据给出的数据，分析数据并实现以下要求：

1、统计各个州的捐款总额，并在美国地图上画出各州捐款总额的热度图（heatmap），颜色越深的州代表捐款额越多，要求图表美观易懂。

2、统计获得捐赠额最多的三位候选人的捐赠额变化趋势，使用折线图展示，横轴表示时间，纵轴表示捐赠额，要求图表美观易懂。

3、分析出获得捐款额最多的候选人，然后将此候选人的捐赠者的姓名画成词云图。

In [1]:
import pandas as pd
import numpy as np
originData = pd.read_csv(
    "data/itcont_2020_20200722_20200820.txt", sep='|')


C:\Users\Delta\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10,15,16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
originData.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00701888,A,M8,P,202009149267170153,10,IND,"BAUMGART, STEPHEN",LOS GATOS,CA,...,VARIAN MEDICAL SYSTEMS,SYSTEMS ENGINEER PHYSICIST,7222020,500,NaN,12885178,1437664,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4092320201849287519
1,C00701888,A,M8,P,202009149267170154,10,IND,"HELBERG, JACOB",SAN FRANCISCO,CA,...,STANFORD CYBER POLICY CENTER,SENIOR ADVISOR,7232020,250000,NaN,12854698,1437664,NaN,NaN,4092320201849287521
2,C00701888,A,M8,P,202009149267170135,10,IND,"CONSIGLI, ANTHONY",MILFORD,MA,...,"WHITEWOOD VENTURES, LLC",CHIEF EXECUTIVE OFFICER,7312020,25000,NaN,14260296,1437664,X,NaN,4092320201849287464
3,C00701888,A,M8,P,202009149267170136,10,IND,"POTASH, ANDREW",MAMARONECK,NY,...,DISTINGUISHED PROGRAMS INSURANCE,CHIEF EXECUTIVE OFFICER,7282020,100000,NaN,12885744,1437664,NaN,NaN,4092320201849287468
4,C00701888,A,M8,P,202009149267170155,10,IND,"WEINER, STEPHEN R.",BOSTON,MA,...,S.R. WEINER AND ASSOCIATES,DEVELOPER,7282020,25000,NaN,12885619,1437664,NaN,NaN,4092320201849287525


In [3]:
data = originData[['CMTE_ID','TRANSACTION_AMT','TRANSACTION_DT','NAME']] # MMDDYYYY
data.head()

,CMTE_ID,TRANSACTION_AMT,TRANSACTION_DT,NAME
0,C00701888,500,7222020,"BAUMGART, STEPHEN"
1,C00701888,250000,7232020,"HELBERG, JACOB"
2,C00701888,25000,7312020,"CONSIGLI, ANTHONY"
3,C00701888,100000,7282020,"POTASH, ANDREW"
4,C00701888,25000,7282020,"WEINER, STEPHEN R."


In [4]:
cclData = pd.read_csv("data/ccl.txt",sep='|')[['CAND_ID','CMTE_ID']]

In [5]:
cclData.head()

cclData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CAND_ID  7082 non-null   object
 1   CMTE_ID  7082 non-null   object
dtypes: object(2)
memory usage: 110.8+ KB


In [6]:
td = data.merge(cclData)
td.head()

,CMTE_ID,TRANSACTION_AMT,TRANSACTION_DT,NAME,CAND_ID
0,C00750745,500,7222020,"BEAUREGARD, STEVE",P00016550
1,C00750745,750,7222020,"EINSTEIN, GORDON",P00016550
2,C00750745,2800,7232020,"ETTINGER, ADAM",P00016550
3,C00750745,2800,7282020,"GROSS, MOSES",P00016550
4,C00750745,2800,7242020,"KAVANAUGH, RYAN",P00016550


In [7]:
dname = pd.read_csv("data/weball20.txt",sep='|')[['CAND_ID','CAND_NAME']]

In [8]:
td = td.merge(dname)
print(td.info())
td['TRANSACTION_DT'] = pd.to_datetime(td['TRANSACTION_DT'],format='%m%d%Y')
td.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 756067 entries, 0 to 756066
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   CMTE_ID          756067 non-null  object
 1   TRANSACTION_AMT  756067 non-null  int64 
 2   TRANSACTION_DT   756067 non-null  int64 
 3   NAME             756067 non-null  object
 4   CAND_ID          756067 non-null  object
 5   CAND_NAME        756067 non-null  object
dtypes: int64(2), object(4)
memory usage: 40.4+ MB
None


,CMTE_ID,TRANSACTION_AMT,TRANSACTION_DT,NAME,CAND_ID,CAND_NAME
0,C00750745,500,2020-07-22,"BEAUREGARD, STEVE",P00016550,"PIERCE, BROCK"
1,C00750745,750,2020-07-22,"EINSTEIN, GORDON",P00016550,"PIERCE, BROCK"
2,C00750745,2800,2020-07-23,"ETTINGER, ADAM",P00016550,"PIERCE, BROCK"
3,C00750745,2800,2020-07-28,"GROSS, MOSES",P00016550,"PIERCE, BROCK"
4,C00750745,2800,2020-07-24,"KAVANAUGH, RYAN",P00016550,"PIERCE, BROCK"


In [9]:
std = td.groupby('CAND_NAME')['TRANSACTION_AMT'].sum().sort_values(ascending=False)

In [21]:
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
%matplotlib

for member in (std.index[:3]):
    tmp = td[td['CAND_NAME'] == member]
    moneyPerDay = tmp.groupby('TRANSACTION_DT')['TRANSACTION_AMT'].sum()
    money = np.cumsum(moneyPerDay)
    money.index.name = 'Date'

    # display(money.index)
    plt.plot(money,label=member)
    
plt.tight_layout()
plt.legend()
# fig = plt.figure(figsize=(40,4))
plt.tick_params(axis='x', labelsize=8) 
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
plt.xlabel(u'日期')
plt.ylabel(u"获得捐赠数额（美元）")

Using matplotlib backend: Qt5Agg


Text(65.14422035217285, 0.5, '获得捐赠数额（美元）')